# Northwind Traders - Data Analysis Challenge

### This notebook analyzes the Northwind database to identify key performance indicators and generate insights to help increase average ticket size and reduce customer churn.



## 1. Setup and Data Loading

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

In [2]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('ggplot')
sns.set(style="whitegrid")

In [3]:
# Define file paths
base_path = "./northwind/"

# Load all CSV files
categories = pd.read_csv(f'{base_path}categories.csv')
customers = pd.read_csv(f'{base_path}customers.csv')
customer_demographics = pd.read_csv(f'{base_path}customer_demographics.csv')
customer_customer_demo = pd.read_csv(f'{base_path}customer_customer_demo.csv')
employees = pd.read_csv(f'{base_path}employees.csv')
employee_territories = pd.read_csv(f'{base_path}employee_territories.csv')
orders = pd.read_csv(f'{base_path}orders.csv')
order_details = pd.read_csv(f'{base_path}order_details.csv')
products = pd.read_csv(f'{base_path}products.csv')
region = pd.read_csv(f'{base_path}region.csv')
shippers = pd.read_csv(f'{base_path}shippers.csv')
suppliers = pd.read_csv(f'{base_path}suppliers.csv')
territories = pd.read_csv(f'{base_path}territories.csv')
us_states = pd.read_csv(f'{base_path}us_states.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 2


In [5]:
# Quick look at what we're working with
print("Datasets overview:")
for name, df in [
    ("categories", categories),
    ("customers", customers),
    ("customer_demographics", customer_demographics),
    ("customer_customer_demo", customer_customer_demo),
    ("employees", employees),
    ("employee_territories", employee_territories),
    ("orders", orders),
    ("order_details", order_details),
    ("products", products),
    ("region", region),
    ("shippers", shippers),
    ("suppliers", suppliers),
    ("territories", territories),
    ("us_states", us_states)
]:
    print(f"\n{name}: {df.shape[0]} rows, {df.shape[1]} columns")
    print(df.columns.tolist())

Datasets overview:


NameError: name 'customer_demographics' is not defined

### 2. Data Cleaning and Preparation

In [ ]:
# Convert date columns to datetime in orders
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')
orders['required_date'] = pd.to_datetime(orders['required_date'], errors='coerce')
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'], errors='coerce')

# Check for missing values
print("\nMissing values in orders:")
print(orders.isnull().sum())

# Fill missing shipped_date with appropriate values or mark as not shipped
orders['is_shipped'] = ~orders['shipped_date'].isna()

# Calculate order_value in order_details
order_details['order_value'] = order_details['unit_price'] * order_details['quantity'] * (1 - order_details['discount'])

# Merge orders and order_details for analysis
orders_with_details = pd.merge(
    orders,
    order_details.groupby('order_id').agg(
        total_amount=('order_value', 'sum'),
        unique_products=('product_id', 'nunique'),
        total_quantity=('quantity', 'sum')
    ).reset_index(),
    on='order_id',
    how='left'
)

In [ ]:
# Merge with customers
orders_with_customers = pd.merge(
    orders_with_details,
    customers[['customer_id', 'company_name', 'country', 'region', 'city']],
    on='customer_id',
    how='left'
)

### 3. Analysis Part 1: Revenue and Sales Performance

#### 3.1 Revenue Analysis

In [ ]:
# Calculate monthly revenue
monthly_revenue = orders_with_customers.copy()
monthly_revenue['year_month'] = monthly_revenue['order_date'].dt.strftime('%Y-%m')
monthly_sales = monthly_revenue.groupby('year_month').agg(
    total_revenue=('total_amount', 'sum'),
    order_count=('order_id', 'nunique'),
    avg_order_value=('total_amount', 'mean')
).reset_index()

In [ ]:
# Sorting by date
monthly_sales['year_month'] = pd.to_datetime(monthly_sales['year_month'] + '-01')
monthly_sales = monthly_sales.sort_values('year_month')
monthly_sales['year_month'] = monthly_sales['year_month'].dt.strftime('%Y-%m')

In [6]:
# Plot monthly sales trends
plt.figure(figsize=(14, 6))
plt.plot(monthly_sales['year_month'], monthly_sales['total_revenue'], marker='o', linewidth=2)
plt.title('Monthly Revenue Trend', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('monthly_revenue_trend.png')
plt.close()

NameError: name 'monthly_sales' is not defined

<Figure size 1400x600 with 0 Axes>

#### 3.2 Average Ticket Size Analysis

In [ ]:
# Calculate average ticket size by customer segment
avg_ticket_by_country = orders_with_customers.groupby('country').agg(
    avg_ticket=('total_amount', 'mean'),
    order_count=('order_id', 'count'),
    total_revenue=('total_amount', 'sum')
).reset_index().sort_values('avg_ticket', ascending=False)

In [ ]:
# Plot top 10 countries by average ticket
top_countries = avg_ticket_by_country.nlargest(10, 'avg_ticket')
plt.figure(figsize=(12, 6))
sns.barplot(x='country', y='avg_ticket', data=top_countries)
plt.title('Top 10 Countries by Average Ticket Size', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Average Ticket ($)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('top_countries_avg_ticket.png')
plt.close()

In [ ]:
# Calculate average items per order
avg_items_per_order = orders_with_details['unique_products'].mean()
print(f"\nAverage unique products per order: {avg_items_per_order:.2f}")

### 4. Analysis Part 2: Customer Behavior and Churn Risk

#### 4.1 Customer Frequency and Recency Analysis

In [ ]:
# Calculate frequency and recency for each customer
customer_frequency = orders.groupby('customer_id').agg(
    order_count=('order_id', 'nunique'),
    last_order_date=('order_date', 'max'),
    first_order_date=('order_date', 'min')
).reset_index()

In [ ]:
# Calculate recency (days since last order)
max_date = orders['order_date'].max()
customer_frequency['recency_days'] = (max_date - customer_frequency['last_order_date']).dt.days
customer_frequency['customer_tenure_days'] = (customer_frequency['last_order_date'] - customer_frequency['first_order_date']).dt.days


In [ ]:
# Merge with customer info
customer_analysis = pd.merge(
    customer_frequency,
    customers[['customer_id', 'company_name', 'country']],
    on='customer_id',
    how='left'
)

In [ ]:
# Define churn risk (customers with no orders in the last 90 days)
customer_analysis['churn_risk'] = customer_analysis['recency_days'] > 90

# Calculate churn rate
churn_rate = customer_analysis['churn_risk'].mean() * 100
print(f"\nCustomer churn risk rate: {churn_rate:.2f}%")

In [ ]:
# Calculate average order value by customer
customer_order_value = pd.merge(
    orders_with_details.groupby('customer_id').agg(
        avg_order_value=('total_amount', 'mean'),
        total_spent=('total_amount', 'sum')
    ).reset_index(),
    customer_analysis[['customer_id', 'order_count', 'recency_days', 'churn_risk']],
    on='customer_id',
    how='left'
)

In [ ]:
# Identify high-value customers at risk of churning
high_value_churn = customer_order_value[
    (customer_order_value['avg_order_value'] > customer_order_value['avg_order_value'].median()) &
    (customer_order_value['churn_risk'])
].sort_values('total_spent', ascending=False)

print(f"\nHigh-value customers at risk of churning: {len(high_value_churn)}")

### 5. Analysis Part 3: Product Performance

#### 5.1 Top Performing Products

In [ ]:
# Analyze product performance
product_performance = pd.merge(
    order_details.groupby('product_id').agg(
        total_revenue=('order_value', 'sum'),
        order_count=('order_id', 'nunique'),
        total_quantity=('quantity', 'sum')
    ).reset_index(),
    products[['product_id', 'product_name', 'category_id', 'unit_price']],
    on='product_id',
    how='left'
)

In [ ]:
# Add category names
product_performance = pd.merge(
    product_performance,
    categories[['category_id', 'category_name']],
    on='category_id',
    how='left'
)

In [ ]:


# Sort by revenue
product_performance = product_performance.sort_values('total_revenue', ascending=False)

# Top 10 products by revenue
top_products = product_performance.head(10)
print("\nTop 10 Products by Revenue:")
print(top_products[['product_name', 'total_revenue', 'order_count', 'total_quantity']])


In [ ]:
# Product category analysis
category_performance = product_performance.groupby('category_name').agg(
    total_revenue=('total_revenue', 'sum'),
    product_count=('product_id', 'nunique'),
    avg_price=('unit_price', 'mean')
).reset_index().sort_values('total_revenue', ascending=False)

In [ ]:
# Plot category performance
plt.figure(figsize=(12, 6))
sns.barplot(x='category_name', y='total_revenue', data=category_performance)
plt.title('Revenue by Product Category', fontsize=16)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Total Revenue ($)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('category_revenue.png')
plt.close()

### 6. Cross-Selling and Basket Analysis

#### 6.1 Identify products frequently purchased together

In [ ]:

# First, get all order-product combinations
order_products = order_details[['order_id', 'product_id']].copy()
order_products = pd.merge(
    order_products,
    products[['product_id', 'product_name']],
    on='product_id',
    how='left'
)

In [ ]:
# Create a product pairs table
def get_product_pairs(order_id):
    prods = order_products[order_products['order_id'] == order_id]['product_name'].tolist()
    pairs = []
    for i in range(len(prods)):
        for j in range(i+1, len(prods)):
            # Ensure alphabetical sorting for consistent pairs
            if prods[i] < prods[j]:
                pairs.append((prods[i], prods[j]))
            else:
                pairs.append((prods[j], prods[i]))
    return pairs

In [ ]:
# Get orders with multiple products
orders_with_multiple = orders_with_details[orders_with_details['unique_products'] > 1]['order_id'].unique()

# For efficiency, limit to a subset of orders for this analysis
if len(orders_with_multiple) > 100:
    sample_orders = np.random.choice(orders_with_multiple, 100, replace=False)
else:
    sample_orders = orders_with_multiple

In [ ]:
# Generate all pairs from sample orders
all_pairs = []
for order_id in sample_orders:
    all_pairs.extend(get_product_pairs(order_id))

# Count frequencies
pair_counts = {}
for pair in all_pairs:
    if pair in pair_counts:
        pair_counts[pair] += 1
    else:
        pair_counts[pair] = 1

# Convert to dataframe and sort
pairs_df = pd.DataFrame([(p[0], p[1], c) for p, c in pair_counts.items()],
                       columns=['product1', 'product2', 'frequency'])
pairs_df = pairs_df.sort_values('frequency', ascending=False)

print("\nTop 5 Product Pairs Frequently Purchased Together:")
print(pairs_df.head(5))

### 7. Key Insights and Recommendations

#### 7.1 Average Ticket Size Enhancement

In [ ]:
# Analyze impact of product variety on order value
order_details_with_info = pd.merge(
    order_details,
    products[['product_id', 'product_name', 'category_id']],
    on='product_id',
    how='left'
)

order_details_with_info = pd.merge(
    order_details_with_info,
    categories[['category_id', 'category_name']],
    on='category_id',
    how='left'
)

In [ ]:
# Calculate product and category diversity per order
order_diversity = order_details_with_info.groupby('order_id').agg(
    product_count=('product_id', 'nunique'),
    category_count=('category_id', 'nunique'),
    order_value=('order_value', 'sum')
).reset_index()

# Analyze relationship between product diversity and order value
plt.figure(figsize=(10, 6))
sns.scatterplot(x='product_count', y='order_value', data=order_diversity)
plt.title('Relationship Between Product Diversity and Order Value', fontsize=16)
plt.xlabel('Number of Unique Products in Order', fontsize=12)
plt.ylabel('Total Order Value ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('product_diversity_order_value.png')
plt.close()

In [ ]:
# Calculate correlation
correlation = order_diversity['product_count'].corr(order_diversity['order_value'])
print(f"\nCorrelation between product count and order value: {correlation:.4f}")

#### 7.2 Churn Reduction Strategies

In [ ]:
# Identify patterns in customer behavior before churn
# First, calculate time between orders for each customer
customer_orders = orders[['customer_id', 'order_id', 'order_date']].sort_values(['customer_id', 'order_date'])
customer_orders['prev_order_date'] = customer_orders.groupby('customer_id')['order_date'].shift(1)
customer_orders['days_since_prev_order'] = (customer_orders['order_date'] - customer_orders['prev_order_date']).dt.days

In [ ]:
# Filter out first orders (where prev_order_date is NaN)
customer_orders = customer_orders.dropna(subset=['prev_order_date'])

# Compare active vs churned customers
active_customers = customer_analysis[~customer_analysis['churn_risk']]['customer_id'].tolist()
churned_customers = customer_analysis[customer_analysis['churn_risk']]['customer_id'].tolist()

active_order_gaps = customer_orders[customer_orders['customer_id'].isin(active_customers)]['days_since_prev_order']
churned_order_gaps = customer_orders[customer_orders['customer_id'].isin(churned_customers)]['days_since_prev_order']

In [ ]:
# Calculate average time between orders
if len(active_order_gaps) > 0 and len(churned_order_gaps) > 0:
    print(f"\nAvg days between orders (active customers): {active_order_gaps.mean():.2f}")
    print(f"Avg days between orders (churned customers): {churned_order_gaps.mean():.2f}")

### 8. Executive Summary

In [ ]:
# Calculate key metrics for executive summary
total_revenue = orders_with_details['total_amount'].sum()
total_orders = len(orders_with_details)
avg_order_value = total_revenue / total_orders if total_orders > 0 else 0
total_customers = len(customers)
repeat_customers = customer_analysis[customer_analysis['order_count'] > 1].shape[0]
repeat_rate = repeat_customers / total_customers * 100

print("\n*** EXECUTIVE SUMMARY ***")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Orders: {total_orders}")
print(f"Average Order Value: ${avg_order_value:,.2f}")
print(f"Total Customers: {total_customers}")
print(f"Repeat Purchase Rate: {repeat_rate:.2f}%")
print(f"Customer Churn Risk Rate: {churn_rate:.2f}%")

In [ ]:
# Create recommendations based on analysis
print("\n*** KEY RECOMMENDATIONS ***")
print("1. Implement cross-selling strategies for top product pairs to increase average ticket size")
print("2. Develop a customer retention program targeting high-value customers with high churn risk")
print("3. Expand product categories with highest average order values to improve overall revenue")
print("4. Create a loyalty program to reduce time between orders for customers showing signs of disengagement")
print("5. Implement personalized marketing campaigns based on purchase history and category preferences")

### 9. Data Visualization Dashboard Preparation

In [ ]:
# Prepare data for dashboard visualizations
# Monthly sales trend
monthly_trend = orders.copy()
monthly_trend['year_month'] = monthly_trend['order_date'].dt.strftime('%Y-%m')
monthly_trend = pd.merge(
    monthly_trend,
    order_details.groupby('order_id')['order_value'].sum().reset_index(),
    on='order_id',
    how='left'
)

monthly_summary = monthly_trend.groupby('year_month').agg(
    total_revenue=('order_value', 'sum'),
    order_count=('order_id', 'nunique'),
    customer_count=('customer_id', 'nunique')
).reset_index()

In [ ]:
# Save processed data for potential dashboard use
monthly_summary.to_csv('monthly_sales_summary.csv', index=False)
category_performance.to_csv('category_performance.csv', index=False)
top_products.to_csv('top_products.csv', index=False)
high_value_churn.to_csv('high_value_churn_risk.csv', index=False)

In [ ]:
# Calculate customer segmentation for targeting
customer_segments = customer_order_value.copy()
customer_segments['recency_segment'] = pd.qcut(customer_segments['recency_days'], 3, labels=['Recent', 'Moderate', 'Distant'])
customer_segments['frequency_segment'] = pd.qcut(customer_segments['order_count'], 3, labels=['Low', 'Medium', 'High'])
customer_segments['monetary_segment'] = pd.qcut(customer_segments['total_spent'], 3, labels=['Low', 'Medium', 'High'])


In [ ]:
# Combine into RFM score
segment_map = {'Low': 0, 'Medium': 1, 'High': 2, 'Recent': 2, 'Moderate': 1, 'Distant': 0}
customer_segments['recency_score'] = customer_segments['recency_segment'].map(segment_map)
customer_segments['frequency_score'] = customer_segments['frequency_segment'].map(segment_map)
customer_segments['monetary_score'] = customer_segments['monetary_segment'].map(segment_map)
customer_segments['rfm_score'] = customer_segments['recency_score'] + customer_segments['frequency_score'] + customer_segments['monetary_score']

In [ ]:
# Identify VIP customers and at-risk high-value customers
customer_segments['segment'] = pd.cut(
    customer_segments['rfm_score'],
    bins=[0, 2, 4, 6],
    labels=['Low Value', 'Medium Value', 'High Value']
)

print("\n*** CUSTOMER SEGMENTATION ***")
print(customer_segments['segment'].value_counts())

In [ ]:
# Save for reporting
customer_segments.to_csv('customer_segments.csv', index=False)

### 10. Financial Impact Projections

In [ ]:


# Estimate financial impact of recommendations
# 1. Cross-selling impact
avg_order_value_current = avg_order_value
potential_increase_pct = 0.15  # Estimated 15% increase from cross-selling
potential_avg_order = avg_order_value_current * (1 + potential_increase_pct)
annual_orders_estimate = total_orders * (12 / len(monthly_summary))  # Estimate annual orders
potential_revenue_gain = (potential_avg_order - avg_order_value_current) * annual_orders_estimate


In [ ]:

# 2. Churn reduction impact
high_value_churned = high_value_churn.shape[0]
avg_high_value_spend = high_value_churn['total_spent'].mean()
churn_recovery_rate = 0.3  # Assume we can recover 30% of high-value churned customers
potential_churn_recovery = high_value_churned * churn_recovery_rate * avg_high_value_spend


In [ ]:

# Total potential financial impact
total_potential_impact = potential_revenue_gain + potential_churn_recovery

print("\n*** FINANCIAL IMPACT PROJECTIONS ***")
print(f"Potential Annual Revenue Gain from Cross-selling: ${potential_revenue_gain:,.2f}")
print(f"Potential Revenue Recovery from Churn Reduction: ${potential_churn_recovery:,.2f}")
print(f"Total Potential Financial Impact: ${total_potential_impact:,.2f}")

### 11. Implementation Roadmap

In [ ]:
print("\n*** IMPLEMENTATION ROADMAP ***")
print("Phase 1 (Month 1-2): Data Integration and Dashboard Setup")
print("- Connect all data sources (ERP, Salesforce, ContaAzul)")
print("- Implement automated data pipelines")
print("- Create executive and operational dashboards")
print("\nPhase 2 (Month 3-4): Customer Retention Program")
print("- Implement churn prediction model")
print("- Develop targeted retention campaigns for high-risk customers")
print("- Launch customer loyalty program")
print("\nPhase 3 (Month 5-6): Revenue Optimization")
print("- Implement cross-selling recommendations in ordering system")
print("- Optimize product mix based on margin analysis")
print("- Personalize customer communications based on purchase history")
print("\nPhase 4 (Month 7-8): Advanced Analytics")
print("- Implement predictive analytics for demand forecasting")
print("- Develop customer lifetime value projections")
print("- Create automated anomaly detection for business metrics")